# Cancer Data Experimental ML Analysis v2

In [1]:
#import useful libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
import sys
import umap
import random
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.decomposition import KernelPCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
#import data (beware, it takes around 10 minutes)
df = pd.read_csv('../clinical_TumorCompendium_v11_PolyA_2020-04-09.tsv', sep='\t', header=0)
df2 = pd.read_csv('../TumorCompendium_v11_PolyA_hugo_log2tpm_58581genes_2020-04-09.tsv', sep='\t', header=0)

In [12]:
# *** try to run 6 component clustering to see if certain clustering algos (for carcinoma e.g.) don't work as well as CST
# *** will likely need 6 group samples, cannot visualize easily, so will likely need confusion matrices to view results
# *** reference assignments to see how to use confusion matrices for this application

#experiment method for visualizing performance for PCA, KPCA, and UMAP
def run_experiment_print(g1_samples, g2_samples, g3_samples, g4_samples, g5_samples, g6_samples):
    
    x = pd.concat([g1_samples.T, g2_samples.T, g3_samples.T, g4_samples.T, g5_samples.T, g6_samples.T]) #add together all postiive and negative labels 
    x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

    target_g1 = pd.DataFrame(np.zeros((len(g1_samples.T), 1))) #get number of target group 1
    target_g2 = pd.DataFrame(np.ones((len(g2_samples.T), 1))) #get number of target group 2
    target_g3 = pd.DataFrame(2*np.ones((len(g3_samples.T), 1))) #get number of target group 3
    target_g4 = pd.DataFrame(3*np.ones((len(g4_samples.T), 1))) #get number of target group 4
    target_g5 = pd.DataFrame(4*np.ones((len(g5_samples.T), 1))) #get number of target group 5
    target_g6 = pd.DataFrame(5*np.ones((len(g6_samples.T), 1))) #get number of target group 6

    target = pd.concat([target_g1, target_g2, target_g3, target_g4, target_g5, target_g6]) #concatenate target zeros ones twos etc together
    target = target.reset_index(drop=True)

    #split the data into test and train
    X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=0.3, random_state=42)

    #PCA
    pca = PCA(n_components=6)
    principalComponents = pca.fit_transform(x) #break down x with PCA (dimensionality reduction) into 6 components
    principalDf = pd.DataFrame(data = principalComponents
                 , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'
                             , 'principal component 5', 'principal component 6'])
    finalDf = pd.concat([principalDf, target], axis = 1) #puts labels alongside PCA components

    
    #calculate and display confusion matrix instead (potentially also calculate BAC)
    print(principalDf)
    print()
    print(finalDf)
    
    
#     fig = plt.figure(figsize = (8,8))
#     ax = fig.add_subplot(2,2,1) 
#     ax.set_xlabel('Principal Component 1', fontsize = 15)
#     ax.set_ylabel('Principal Component 2', fontsize = 15)
#     ax.set_title('2 component PCA', fontsize = 20)
#     targets = [1.0, 0.0]
#     colors = ['r', 'g']
#     for target, color in zip(targets,colors):
#         indicesToKeep = finalDf[0] == target
#         ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
#                    , finalDf.loc[indicesToKeep, 'principal component 2']
#                    , c = color
#                    , s = 50)
#     ax.legend(targets)
#     ax.grid()
#     plt.show()

    #KPCA
    pca = KernelPCA(n_components=6, kernel='poly')
    principalComponents = pca.fit_transform(x)
    principalDfk = pd.DataFrame(data = principalComponents
                 , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'
                             , 'principal component 5', 'principal component 6'])
    target = pd.concat([target_g1, target_g2, target_g3, target_g4, target_g5, target_g6])
    target = target.reset_index(drop=True)
    finalDfk = pd.concat([principalDfk, target], axis = 1)
    
    #calculate and display confusion matrix instead (potentially also calculate BAC)
    print(principalDfk)
    print()
    print(finalDfk)
    
    

#     fig = plt.figure(figsize = (8,8))
#     ax = fig.add_subplot(2,2,2) 
#     ax.set_xlabel('Principal Component 1', fontsize = 15)
#     ax.set_ylabel('Principal Component 2', fontsize = 15)
#     ax.set_title('2 component KPCA', fontsize = 20)
#     ax.set_ylim(-0.2,0.0)
#     ax.set_xlim(-0.5,0.5)
#     targets = [1.0, 0.0]
#     colors = ['r', 'g']
#     for target, color in zip(targets,colors):
#         indicesToKeep = finalDfk[0] == target
#         ax.scatter(finalDfk.loc[indicesToKeep, 'principal component 1']
#                    , finalDfk.loc[indicesToKeep, 'principal component 2']
#                    , c = color
#                    , s = 50)
#     ax.legend(targets)
#     ax.grid()
#     plt.show()

    #UMAP
#     target = pd.concat([target_g1, target_g2, target_g3, target_g4, target_g5, target_g6])
#     target = target.reset_index(drop=True)
#     mapper = umap.UMAP(n_neighbors=20).fit(x, target.to_numpy())
#     y = mapper.transform(x)
#     principalDfu = pd.DataFrame(data = y
#                  , columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4'
#                              , 'principal component 5', 'principal component 6'])

#     finalDfu = pd.concat([principalDfu, target], axis = 1)
    
    #calculate and display confusion matrix instead (potentially also calculate BAC)
    
    
    
    
    

#     fig = plt.figure(figsize = (8,8))
#     ax = fig.add_subplot(2,2,3) 
#     ax.set_xlabel('Principal Component 1', fontsize = 15)
#     ax.set_ylabel('Principal Component 2', fontsize = 15)
#     ax.set_title('2 component UMAP', fontsize = 20)
#     targets = [1.0, 0.0]
#     colors = ['r', 'g']
#     for target, color in zip(targets,colors):
#         indicesToKeep = finalDfu[0] == target
#         ax.scatter(finalDfu.loc[indicesToKeep, 'principal component 1']
#                    , finalDfu.loc[indicesToKeep, 'principal component 2']
#                    , c = color
#                    , s = 50)
#     ax.legend(targets)
#     ax.grid()
#     plt.show()
    
    return

## Methods for calculations

In [34]:
def kNN_PCA(X_train, X_test, y_train, y_test, k):
    pca = PCA(n_components = k)
    pca.fit(X_train)
    X_train_trans = pca.transform(X_train)
    X_test_trans = pca.transform(X_test)
    neighbors = KNeighborsClassifier(n_neighbors=6)
    neighbors.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, neighbors.predict(X_test_trans))

In [35]:
def kNN_UMAP(X_train, X_test, y_train, y_test, k):
    mapper = umap.UMAP(n_neighbors=(k*4)).fit(X_train, y_train)
    X_train_trans = mapper.transform(X_train)
    X_test_trans = mapper.transform(X_test)
    neighbors = KNeighborsClassifier(n_neighbors=6)
    neighbors.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, neighbors.predict(X_test_trans))

In [ ]:
def kNN_MI(X_train, X_test, y_train, y_test, k):
    mutual_info_classif()

In [36]:
def kNN_KPCA(X_train, X_test, y_train, y_test, k):
    kpca = KernelPCA(n_components = k, kernel = 'poly')
    kpca.fit(X_train)
    X_train_trans = kpca.transform(X_train)
    X_test_trans = kpca.transform(X_test)
    neighbors = KNeighborsClassifier(n_neighbors=6)
    neighbors.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, neighbors.predict(X_test_trans))

In [37]:
def SVM_PCA(X_train, X_test, y_train, y_test, k):
    pca = PCA(n_components = k)
    pca.fit(X_train)
    X_train_trans = pca.transform(X_train)
    X_test_trans = pca.transform(X_test)
    svm = SVC(kernel='linear', C=1)
    svm.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, svm.predict(X_test_trans))

In [38]:
def SVM_UMAP(X_train, X_test, y_train, y_test, k):
    mapper = umap.UMAP(n_neighbors=(k*4)).fit(X_train, y_train)
    X_train_trans = mapper.transform(X_train)
    X_test_trans = mapper.transform(X_test)
    svm = SVC(kernel='linear', C=1)
    svm.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, svm.predict(X_test_trans))

In [ ]:
def SVM_MI(X_train, X_test, y_train, y_test, k):
    

In [39]:
def SVM_KPCA(X_train, X_test, y_train, y_test, k):
    kpca = KernelPCA(n_components = k, kernel = 'poly')
    kpca.fit(X_train)
    X_train_trans = kpca.transform(X_train)
    X_test_trans = kpca.transform(X_test)
    svm = SVC(kernel='linear', C=1)
    svm.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, svm.predict(X_test_trans))

In [40]:
def GNB_PCA(X_train, X_test, y_train, y_test, k):
    pca = PCA(n_components = k)
    pca.fit(X_train)
    X_train_trans = pca.transform(X_train)
    X_test_trans = pca.transform(X_test)
    gnb = GaussianNB()
    gnb.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, gnb.predict(X_test_trans))

In [41]:
def GNB_UMAP(X_train, X_test, y_train, y_test, k):
    mapper = umap.UMAP(n_neighbors=(k*4)).fit(X_train, y_train)
    X_train_trans = mapper.transform(X_train)
    X_test_trans = mapper.transform(X_test)
    gnb = GaussianNB()
    gnb.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, gnb.predict(X_test_trans))

In [ ]:
def GNB_MI(X_train, X_test, y_train, y_test, k):
    

In [42]:
def GNB_KPCA(X_train, X_test, y_train, y_test, k):
    kpca = KernelPCA(n_components = k, kernel = 'poly')
    kpca.fit(X_train)
    X_train_trans = kpca.transform(X_train)
    X_test_trans = kpca.transform(X_test)
    gnb = GaussianNB()
    gnb.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, gnb.predict(X_test_trans))

In [43]:
def RF_PCA(X_train, X_test, y_train, y_test, k):
    pca = PCA(n_components = k)
    pca.fit(X_train)
    X_train_trans = pca.transform(X_train)
    X_test_trans = pca.transform(X_test)
    rf = RandomForestClassifier(random_state=137)
    rf.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, rf.predict(X_test_trans))

In [44]:
def RF_UMAP(X_train, X_test, y_train, y_test, k):
    mapper = umap.UMAP(n_neighbors=(k*4)).fit(X_train, y_train)
    X_train_trans = mapper.transform(X_train)
    X_test_trans = mapper.transform(X_test)
    rf = RandomForestClassifier(random_state=137)
    rf.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, rf.predict(X_test_trans))

In [ ]:
def RF_MI(X_train, X_test, y_train, y_test, k):
    

In [45]:
def RF_KPCA(X_train, X_test, y_train, y_test, k):
    kpca = KernelPCA(n_components = k, kernel = 'poly')
    kpca.fit(X_train)
    X_train_trans = kpca.transform(X_train)
    X_test_trans = kpca.transform(X_test)
    rf = RandomForestClassifier(random_state=137)
    rf.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, rf.predict(X_test_trans))

In [46]:
def SGD_PCA(X_train, X_test, y_train, y_test, k):
    pca = PCA(n_components = k)
    pca.fit(X_train)
    X_train_trans = pca.transform(X_train)
    X_test_trans = pca.transform(X_test)
    sgd = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))
    sgd.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, sgd.predict(X_test_trans))

In [47]:
def SGD_UMAP(X_train, X_test, y_train, y_test, k):
    mapper = umap.UMAP(n_neighbors=(k*4)).fit(X_train, y_train)
    X_train_trans = mapper.transform(X_train)
    X_test_trans = mapper.transform(X_test)
    sgd = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))
    sgd.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, sgd.predict(X_test_trans))

In [ ]:
def SGD_MI(X_train, X_test, y_train, y_test, k):
    

In [48]:
def SGD_KPCA(X_train, X_test, y_train, y_test, k):
    kpca = KernelPCA(n_components = k, kernel = 'poly')
    kpca.fit(X_train)
    X_train_trans = kpca.transform(X_train)
    X_test_trans = kpca.transform(X_test)
    sgd = make_pipeline(StandardScaler(), SGDClassifier(max_iter=1000, tol=1e-3))
    sgd.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, sgd.predict(X_test_trans))

In [49]:
def LDA_PCA(X_train, X_test, y_train, y_test, k):
    pca = PCA(n_components = k)
    pca.fit(X_train)
    X_train_trans = pca.transform(X_train)
    X_test_trans = pca.transform(X_test)
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, lda.predict(X_test_trans))

In [50]:
def LDA_UMAP(X_train, X_test, y_train, y_test, k):
    mapper = umap.UMAP(n_neighbors=(k*4)).fit(X_train, y_train)
    X_train_trans = mapper.transform(X_train)
    X_test_trans = mapper.transform(X_test)
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, lda.predict(X_test_trans))

In [ ]:
def LDA_MI(X_train, X_test, y_train, y_test, k):
    

In [51]:
def LDA_KPCA(X_train, X_test, y_train, y_test, k):
    kpca = KernelPCA(n_components = k, kernel = 'poly')
    kpca.fit(X_train)
    X_train_trans = kpca.transform(X_train)
    X_test_trans = kpca.transform(X_test)
    lda = LinearDiscriminantAnalysis()
    lda.fit(X_train_trans, y_train)
    return balanced_accuracy_score(y_test, lda.predict(X_test_trans))

In [59]:
def experiment_samples6(g1_samples, g2_samples, g3_samples, g4_samples, g5_samples, g6_samples):
    x = pd.concat([g1_samples.T, g2_samples.T, g3_samples.T, g4_samples.T, g5_samples.T, g6_samples.T]) #add together all postiive and negative labels 
    x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

    target_g1 = pd.DataFrame(np.zeros((len(g1_samples.T), 1))) #get number of target group 1
    target_g2 = pd.DataFrame(np.ones((len(g2_samples.T), 1))) #get number of target group 2
    target_g3 = pd.DataFrame(2*np.ones((len(g3_samples.T), 1))) #get number of target group 3
    target_g4 = pd.DataFrame(3*np.ones((len(g4_samples.T), 1))) #get number of target group 4
    target_g5 = pd.DataFrame(4*np.ones((len(g5_samples.T), 1))) #get number of target group 5
    target_g6 = pd.DataFrame(5*np.ones((len(g6_samples.T), 1))) #get number of target group 6

    target = pd.concat([target_g1, target_g2, target_g3, target_g4, target_g5, target_g6]) #concatenate target zeros ones twos etc together
    target = target.reset_index(drop=True)

    #split the data into test and train
    X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=0.3, random_state=42)
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    
    kNN_PCA_BAC_val = kNN_PCA(X_train, X_test, y_train, y_test, 6)
    print("kNN PCA BAC value is:", kNN_PCA_BAC_val)
    
    kNN_UMAP_BAC_val = kNN_UMAP(X_train, X_test, y_train, y_test, 6)
    print("kNN UMAP BAC value is:", kNN_UMAP_BAC_val)
    
    kNN_KPCA_BAC_val = kNN_KPCA(X_train, X_test, y_train, y_test, 6)
    print("kNN KPCA BAC value is:", kNN_KPCA_BAC_val)
    
    SVM_PCA_BAC_val = SVM_PCA(X_train, X_test, y_train, y_test, 6)
    print("SVM PCA BAC value is:", SVM_PCA_BAC_val)
    
    SVM_UMAP_BAC_val = SVM_UMAP(X_train, X_test, y_train, y_test, 6)
    print("SVM UMAP BAC value is:", SVM_UMAP_BAC_val)
    
    SVM_KPCA_BAC_val = SVM_KPCA(X_train, X_test, y_train, y_test, 6)
    print("SVM KPCA BAC value is:", SVM_KPCA_BAC_val)
    
    SGD_PCA_BAC_val = SGD_PCA(X_train, X_test, y_train, y_test, 6)
    print("SGD PCA BAC value is:", SGD_PCA_BAC_val)
    
    SGD_UMAP_BAC_val = SGD_UMAP(X_train, X_test, y_train, y_test, 6)
    print("SGD UMAP BAC value is:", SGD_UMAP_BAC_val)
    
    SGD_KPCA_BAC_val = SGD_KPCA(X_train, X_test, y_train, y_test, 6)
    print("SGD KPCA BAC value is:", SGD_KPCA_BAC_val)
    
    GNB_PCA_BAC_val = GNB_PCA(X_train, X_test, y_train, y_test, 6)
    print("GNB PCA BAC value is:", GNB_PCA_BAC_val)
    
    GNB_UMAP_BAC_val = GNB_UMAP(X_train, X_test, y_train, y_test, 6)
    print("GNB UMAP BAC value is:", GNB_UMAP_BAC_val)
    
    GNB_KPCA_BAC_val = GNB_KPCA(X_train, X_test, y_train, y_test, 6)
    print("GNB KPCA BAC value is:", GNB_KPCA_BAC_val)
    
    RF_PCA_BAC_val = RF_PCA(X_train, X_test, y_train, y_test, 6)
    print("RF PCA BAC value is:", RF_PCA_BAC_val)
    
    RF_UMAP_BAC_val = RF_UMAP(X_train, X_test, y_train, y_test, 6)
    print("RF UMAP BAC value is:", RF_UMAP_BAC_val)
    
    RF_KPCA_BAC_val = RF_KPCA(X_train, X_test, y_train, y_test, 6)
    print("RF KPCA BAC value is:", RF_KPCA_BAC_val)
    
    LDA_PCA_BAC_val = LDA_PCA(X_train, X_test, y_train, y_test, 6)
    print("LDA PCA BAC value is:", LDA_PCA_BAC_val)
    
    LDA_UMAP_BAC_val = LDA_UMAP(X_train, X_test, y_train, y_test, 6)
    print("LDA UMAP BAC value is:", LDA_UMAP_BAC_val)
    
    LDA_KPCA_BAC_val = LDA_KPCA(X_train, X_test, y_train, y_test, 6)
    print("LDA KPCA BAC value is:", LDA_KPCA_BAC_val)
    
    
    
    return

The sample groupings used below contain 6 different types of carcinomas

In [8]:
samples1 = df[df['disease']=='lung squamous cell carcinoma']['th_sampleid'] 
g1_samples = df2.loc[:,list(samples1)]

samples2 = df[df['disease']=='kidney clear cell carcinoma']['th_sampleid'] 
g2_samples = df2.loc[:,list(samples2)]

samples3 = df[df['disease']=='thyroid carcinoma']['th_sampleid'] 
g3_samples = df2.loc[:,list(samples3)]

samples4 = df[df['disease']=='head & neck squamous cell carcinoma']['th_sampleid'] 
g4_samples = df2.loc[:,list(samples4)]

samples5 = df[df['disease']=='kidney papillary cell carcinoma']['th_sampleid'] 
g5_samples = df2.loc[:,list(samples5)]

samples6 = df[df['disease']=='bladder urothelial carcinoma']['th_sampleid'] 
g6_samples = df2.loc[:,list(samples6)]

In [60]:
experiment_samples6(g1_samples, g2_samples, g3_samples, g4_samples, g5_samples, g6_samples)

kNN PCA BAC value is: 0.8191735605831981
kNN UMAP BAC value is: 0.8781588556851182
kNN KPCA BAC value is: 0.6659747032053845
SVM PCA BAC value is: 0.8605380988419928
SVM UMAP BAC value is: 0.8703463697712124
SVM KPCA BAC value is: 0.254651287309182
SGD PCA BAC value is: 0.7927727939042741
SGD UMAP BAC value is: 0.8746813454123733
SGD KPCA BAC value is: 0.3429227175700433
GNB PCA BAC value is: 0.7792619153313404
GNB UMAP BAC value is: 0.8738679009230412
GNB KPCA BAC value is: 0.21724243445825608
RF PCA BAC value is: 0.8452397371201684
RF UMAP BAC value is: 0.8602572939458281
RF KPCA BAC value is: 0.7345518531022049
LDA PCA BAC value is: 0.8271968103861534
LDA UMAP BAC value is: 0.8823939679191627
LDA KPCA BAC value is: 0.1785571884256095


In [55]:
def experiment_samples4(g1_samples, g2_samples, g3_samples, g4_samples):
    x = pd.concat([g1_samples.T, g2_samples.T, g3_samples.T, g4_samples.T]) #add together all postiive and negative labels 
    x = StandardScaler().fit_transform(x) #standardize values in x (z-score standardization)

    target_g1 = pd.DataFrame(np.zeros((len(g1_samples.T), 1))) #get number of target group 1
    target_g2 = pd.DataFrame(np.ones((len(g2_samples.T), 1))) #get number of target group 2
    target_g3 = pd.DataFrame(2*np.ones((len(g3_samples.T), 1))) #get number of target group 3
    target_g4 = pd.DataFrame(3*np.ones((len(g4_samples.T), 1))) #get number of target group 4

    target = pd.concat([target_g1, target_g2, target_g3, target_g4]) #concatenate target zeros ones twos etc together
    target = target.reset_index(drop=True)

    #split the data into test and train
    X_train, X_test, y_train, y_test = train_test_split(x, target.to_numpy(), test_size=0.3, random_state=42)
    y_train = y_train.ravel()
    y_test = y_test.ravel()
    
    kNN_PCA_BAC_val = kNN_PCA(X_train, X_test, y_train, y_test, 4)
    print("kNN PCA BAC value is:", kNN_PCA_BAC_val)
    
    kNN_UMAP_BAC_val = kNN_UMAP(X_train, X_test, y_train, y_test, 4)
    print("kNN UMAP BAC value is:", kNN_UMAP_BAC_val)
    
    kNN_KPCA_BAC_val = kNN_KPCA(X_train, X_test, y_train, y_test, 4)
    print("kNN KPCA BAC value is:", kNN_KPCA_BAC_val)
    
    SVM_PCA_BAC_val = SVM_PCA(X_train, X_test, y_train, y_test, 4)
    print("SVM PCA BAC value is:", SVM_PCA_BAC_val)
    
    SVM_UMAP_BAC_val = SVM_UMAP(X_train, X_test, y_train, y_test, 4)
    print("SVM UMAP BAC value is:", SVM_UMAP_BAC_val)
    
    SVM_KPCA_BAC_val = SVM_KPCA(X_train, X_test, y_train, y_test, 4)
    print("SVM KPCA BAC value is:", SVM_KPCA_BAC_val)
    
    SGD_PCA_BAC_val = SGD_PCA(X_train, X_test, y_train, y_test, 4)
    print("SGD PCA BAC value is:", SGD_PCA_BAC_val)
    
    SGD_UMAP_BAC_val = SGD_UMAP(X_train, X_test, y_train, y_test, 4)
    print("SGD UMAP BAC value is:", SGD_UMAP_BAC_val)
    
    SGD_KPCA_BAC_val = SGD_KPCA(X_train, X_test, y_train, y_test, 4)
    print("SGD KPCA BAC value is:", SGD_KPCA_BAC_val)
    
    GNB_PCA_BAC_val = GNB_PCA(X_train, X_test, y_train, y_test, 4)
    print("GNB PCA BAC value is:", GNB_PCA_BAC_val)
    
    GNB_UMAP_BAC_val = GNB_UMAP(X_train, X_test, y_train, y_test, 4)
    print("GNB UMAP BAC value is:", GNB_UMAP_BAC_val)
    
    GNB_KPCA_BAC_val = GNB_KPCA(X_train, X_test, y_train, y_test, 4)
    print("GNB KPCA BAC value is:", GNB_KPCA_BAC_val)
    
    RF_PCA_BAC_val = RF_PCA(X_train, X_test, y_train, y_test, 4)
    print("RF PCA BAC value is:", RF_PCA_BAC_val)
    
    RF_UMAP_BAC_val = RF_UMAP(X_train, X_test, y_train, y_test, 4)
    print("RF UMAP BAC value is:", RF_UMAP_BAC_val)
    
    RF_KPCA_BAC_val = RF_KPCA(X_train, X_test, y_train, y_test, 4)
    print("RF KPCA BAC value is:", RF_KPCA_BAC_val)
    
    LDA_PCA_BAC_val = LDA_PCA(X_train, X_test, y_train, y_test, 4)
    print("LDA PCA BAC value is:", LDA_PCA_BAC_val)
    
    LDA_UMAP_BAC_val = LDA_UMAP(X_train, X_test, y_train, y_test, 4)
    print("LDA UMAP BAC value is:", LDA_UMAP_BAC_val)
    
    LDA_KPCA_BAC_val = LDA_KPCA(X_train, X_test, y_train, y_test, 4)
    print("LDA KPCA BAC value is:", LDA_KPCA_BAC_val)
    
    
    
    return

The sample groupings used below contain 4 different types of adenocarcinomas

In [56]:
samples7 = df[df['disease']=='lung adenocarcinoma']['th_sampleid'] 
g7_samples = df2.loc[:,list(samples7)]

samples8 = df[df['disease']=='stomach adenocarcinoma']['th_sampleid'] 
g8_samples = df2.loc[:,list(samples8)]

samples9 = df[df['disease']=='prostate adenocarcinoma']['th_sampleid'] 
g9_samples = df2.loc[:,list(samples9)]

samples10 = df[df['disease']=='colon adenocarcinoma']['th_sampleid'] 
g10_samples = df2.loc[:,list(samples10)]

In [57]:
experiment_samples4(g7_samples, g8_samples, g9_samples, g10_samples)

kNN PCA BAC value is: 0.9111368513402256
kNN UMAP BAC value is: 0.8785311887165184
kNN KPCA BAC value is: 0.7237906778929784
SVM PCA BAC value is: 0.9020677961829917
SVM UMAP BAC value is: 0.8756522902769743
SVM KPCA BAC value is: 0.27455404828028196
SGD PCA BAC value is: 0.8688083430746816
SGD UMAP BAC value is: 0.8805473177487764
SGD KPCA BAC value is: 0.25
GNB PCA BAC value is: 0.7397709250855056
GNB UMAP BAC value is: 0.8708271724806713
GNB KPCA BAC value is: 0.2690098809675259
RF PCA BAC value is: 0.9247474850519312
RF UMAP BAC value is: 0.8809078176419616
RF KPCA BAC value is: 0.7757488965420183
LDA PCA BAC value is: 0.8944356398925611
LDA UMAP BAC value is: 0.8869562047387358
LDA KPCA BAC value is: 0.25201612903225806
